In [1]:
# BLOCK 1 — Setup & Data (stable, safe normalization, and correct autocast setup)

import os, math, random, time, pickle, json
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torch.backends import cudnn
from torch import optim, GradScaler
from torchvision.transforms import v2

# ✅ Correct device and autocast setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
enable_half = device.type == "cuda"
scaler = GradScaler(enabled=enable_half)
print("Grad scaler is enabled:", enable_half)
print("Device:", device)

# ✅ Stable seed & deterministic control
seed = 1337
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device.type == "cuda":
    torch.cuda.manual_seed_all(seed)
cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# === File paths ===
if os.path.exists("/kaggle/input") and os.path.exists("/kaggle/working"):
    SVHN_test = "/kaggle/input/fii-atnn-2025-competition-2/SVHN_test.pkl"
    SVHN_train = "/kaggle/input/fii-atnn-2025-competition-2/SVHN_train.pkl"
    STATS_PATH = "/kaggle/working/svhn_stats.json"
else:
    SVHN_test = "data/SVHN_test.pkl"
    SVHN_train = "data/SVHN_train.pkl"
    STATS_PATH = "svhn_stats.json"


# === Dataset classes ===
class SVHN_Dataset(Dataset):
    def __init__(self, train: bool, transforms: v2.Transform):
        path = SVHN_train if train else SVHN_test
        with open(path, "rb") as fd:
            self.data = pickle.load(fd)
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i: int):
        image, label = self.data[i]
        if self.transforms is None:
            return image, label
        x = self.transforms(image)
        if torch.isnan(x).any() or torch.isinf(x).any():
            # ✅ Skip bad samples
            x = torch.clamp(x, 0, 1)
        return x, label


class TransformedSubset(torch.utils.data.Subset):
    def __init__(self, subset, transform):
        super().__init__(subset.dataset, subset.indices)
        self.transform = transform

    def __getitem__(self, idx):
        image, label = self.dataset[self.indices[idx]]
        if self.transform is None:
            return image, label
        x = self.transform(image)
        if torch.isnan(x).any() or torch.isinf(x).any():
            x = torch.clamp(x, 0, 1)
        return x, label


# === Compute or load dataset mean/std safely ===
def compute_mean_std_from_dataset(dataset):
    to_tensor = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
    n_pixels = 0
    sum_c = torch.zeros(3, dtype=torch.float64)
    sumsq_c = torch.zeros(3, dtype=torch.float64)

    for i in tqdm(range(len(dataset)), desc="Computing mean/std", leave=False):
        img, _ = dataset[i]
        t = to_tensor(img)
        c, h, w = t.shape
        n = h * w
        n_pixels += n
        t = t.to(torch.float64)
        sum_c += t.reshape(c, -1).sum(dim=1)
        sumsq_c += (t.reshape(c, -1) ** 2).sum(dim=1)

    mean = (sum_c / n_pixels).to(torch.float32)
    var = (sumsq_c / n_pixels) - (mean.to(torch.float64) ** 2)
    std = torch.sqrt(var.clamp_min(1e-6)).to(torch.float32)
    return mean.tolist(), std.tolist()


if os.path.exists(STATS_PATH):
    with open(STATS_PATH, "r") as f:
        stats = json.load(f)
    mean, std = stats["mean"], stats["std"]
    print(f"Loaded stats: mean={mean}, std={std}")
else:
    base_train_for_stats = SVHN_Dataset(train=True, transforms=None)
    mean, std = compute_mean_std_from_dataset(base_train_for_stats)
    with open(STATS_PATH, "w") as f:
        json.dump({"mean": mean, "std": std}, f)
    print(f"Computed stats: mean={mean}, std={std}")

# ✅ Sanity check and clamp bad stats
mean = tuple(float(m) for m in mean)
std = tuple(max(float(s), 1e-3) for s in std)
print(f"Using mean={mean}, std={std}")


# === Augmentations ===
train_transforms = v2.Compose([
    v2.ToImage(),
    v2.RandomCrop(32, padding=4, padding_mode="reflect"),
    v2.RandomAffine(degrees=10, translate=(0.12, 0.12), scale=(0.9, 1.1)),
    v2.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.07),
    v2.ToDtype(torch.float32, scale=True),  # ✅ single scaling only
    v2.Normalize(mean, std, inplace=True),
    v2.RandomErasing(p=0.4, scale=(0.02, 0.15), ratio=(0.3, 3.3), inplace=True),
    v2.RandAugment(num_ops=2, magnitude=9),
])

eval_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),  # ✅ only one scaling op
    v2.Normalize(mean, std, inplace=True),
])


# === Dataset and loaders ===
base_train = SVHN_Dataset(train=True, transforms=None)
val_ratio = 0.10
val_len = int(len(base_train) * val_ratio)
train_len = len(base_train) - val_len
train_subset, val_subset = random_split(base_train, [train_len, val_len], generator=torch.Generator().manual_seed(seed))

train_set = TransformedSubset(train_subset, train_transforms)
val_set = TransformedSubset(val_subset, eval_transforms)
test_set = SVHN_Dataset(train=False, transforms=eval_transforms)

BATCH_SIZE = 256
NUM_WORKERS = min(4, (os.cpu_count() or 2) // 2)
pin = device.type == "cuda"

train_loader = DataLoader(
    train_set, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=pin, persistent_workers=NUM_WORKERS > 0, drop_last=True
)
val_loader = DataLoader(
    val_set, batch_size=512, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=pin, persistent_workers=NUM_WORKERS > 0
)
test_loader = DataLoader(
    test_set, batch_size=1024, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=pin, persistent_workers=NUM_WORKERS > 0
)


Grad scaler is enabled: True
Device: cuda
Loaded stats: mean=[0.5070751905441284, 0.48654890060424805, 0.44091787934303284], std=[0.2673342823982239, 0.2564384639263153, 0.2761504650115967]
Using mean=(0.5070751905441284, 0.48654890060424805, 0.44091787934303284), std=(0.2673342823982239, 0.2564384639263153, 0.2761504650115967)


In [2]:
# BLOCK 2 — Model and optimizer updates (stable setup with safe learning dynamics)

import math
from torch.optim.swa_utils import AveragedModel, SWALR

# === Model (unchanged) ===
class VGG13(nn.Module):
    def __init__(self):
        super(VGG13, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),
            nn.Linear(512, 100)
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.layers(x)


# === Experiment configurations (safe & diverse) ===
experiments = [
    # 1️⃣ Baseline — balanced setup
    {
        "name": "exp1_baseline",
        "base_lr": 0.15,
        "ema_decay": 0.9995,
        "rand_mag": 9,
        "mix_p": 0.8,
        "extra_aug": False,
        "remove_ema": False,
        "wd": 5e-4,
    },

    # 2️⃣ Strong augmentation — robustness-focused
    {
        "name": "exp2_strong_aug",
        "base_lr": 0.12,
        "ema_decay": 0.9995,
        "rand_mag": 14,
        "mix_p": 0.8,
        "extra_aug": True,
        "remove_ema": False,
        "wd": 5e-4,
    },

    # 3️⃣ Fast optimization — slightly higher LR but stable decay
    {
        "name": "exp3_fast_opt",
        "base_lr": 0.10,          
        "ema_decay": 0.9975,      
        "rand_mag": 9,
        "mix_p": 0.6,
        "extra_aug": False,
        "remove_ema": False,
        "wd": 6e-4,
    },

    # 4️⃣ Weak augmentation + light regularization
    {
        "name": "exp4_light_aug_fast_lr",
        "base_lr": 0.10,          
        "ema_decay": 0.998,
        "rand_mag": 6,
        "mix_p": 0.5,
        "extra_aug": False,
        "remove_ema": False,
        "wd": 5e-4,
    },
]


# === Global training constants ===
criterion = nn.CrossEntropyLoss(label_smoothing=0.02)

EPOCHS = 100
warmup_pct = 0.20             
min_lr_ratio = 1e-4
grad_clip = 0.5               
patience = 20
SWA_START_RATIO = 0.75        

# ✅ For reproducibility
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True


In [3]:
# BLOCK 3 — Training Loop (stabilized: FP16-safe, NaN-guarded, consistent EMA/SWA updates)

from torch.optim.swa_utils import update_bn
import os, torch

def ensure_nchw(x: torch.Tensor) -> torch.Tensor:
    if x.ndim == 4 and x.shape[1] != 3 and x.shape[-1] == 3:
        return x.permute(0, 3, 1, 2).contiguous()
    return x


def accuracy(outputs: torch.Tensor, targets: torch.Tensor) -> float:
    with torch.no_grad():
        preds = outputs.argmax(1)
        return (preds == targets).float().mean().item()


# === Mixup / CutMix helpers ===
def rand_bbox(W, H, lam):
    cut_rat = (1.0 - lam) ** 0.5
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx, cy = np.random.randint(W), np.random.randint(H)
    x1, y1 = np.clip(cx - cut_w // 2, 0, W), np.clip(cy - cut_h // 2, 0, H)
    x2, y2 = np.clip(cx + cut_w // 2, 0, W), np.clip(cy + cut_h // 2, 0, H)
    return x1, y1, x2, y2


def apply_mixup_cutmix(inputs, targets, mixup_alpha=0.4, cutmix_alpha=1.0, p=0.9):
    if np.random.rand() > p:
        return inputs, (targets, targets, 1.0), None
    bs, c, h, w = inputs.size()
    idx = torch.randperm(bs, device=inputs.device)
    y_a, y_b = targets, targets[idx]
    if np.random.rand() < 0.5:
        lam = np.random.beta(mixup_alpha, mixup_alpha)
        mixed = inputs * lam + inputs[idx] * (1.0 - lam)
        return mixed, (y_a, y_b, float(lam)), "mixup"
    else:
        lam = np.random.beta(cutmix_alpha, cutmix_alpha)
        x1, y1, x2, y2 = rand_bbox(w, h, lam)
        mixed = inputs.clone()
        mixed[:, :, y1:y2, x1:x2] = inputs[idx, :, y1:y2, x1:x2]
        lam = 1.0 - ((x2 - x1) * (y2 - y1) / (w * h + 1e-9))
        return mixed, (y_a, y_b, float(lam)), "cutmix"


def mix_criterion(crit, preds, y_a, y_b, lam):
    return lam * crit(preds, y_a) + (1.0 - lam) * crit(preds, y_b)


# === Stable training step ===
def run_epoch(loader, model, optimizer, scheduler, ema, criterion, train_mode=True, use_swa=False, mix_p=0.9):
    model.train(train_mode)
    total_loss, total_acc, total_n = 0.0, 0.0, 0
    bar = tqdm(loader, leave=False)
    for inputs, targets in bar:
        inputs = ensure_nchw(inputs.to(device, non_blocking=True)).float()
        targets = targets.to(device, non_blocking=True)

        if train_mode:
            inputs_mixed, (y_a, y_b, lam), mode = apply_mixup_cutmix(inputs, targets, p=mix_p)

            with torch.autocast(device_type=device.type, enabled=enable_half):
                outputs = model(inputs_mixed)
                loss = criterion(outputs, targets) if mode is None else mix_criterion(criterion, outputs, y_a, y_b, lam)

            # ✅ skip if NaN/Inf
            if not torch.isfinite(loss):
                print("⚠️  Skipping batch due to NaN/Inf loss")
                optimizer.zero_grad(set_to_none=True)
                continue

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

            if not use_swa:
                scheduler.step()
            if ema is not None:
                ema.update(model)

            batch_acc = accuracy(outputs, targets)

        else:
            with torch.inference_mode(), torch.autocast(device_type=device.type, enabled=enable_half):
                net = ema.ema if ema is not None else model
                outputs = net(inputs)
                loss = criterion(outputs, targets)
            batch_acc = accuracy(outputs, targets)

        bs = targets.size(0)
        total_loss += loss.item() * bs
        total_acc += batch_acc * bs
        total_n += bs
        bar.set_description(f"{'Train' if train_mode else 'Valid'} loss {total_loss/total_n:.4f} acc {total_acc/total_n:.4f}")

    return total_loss / total_n, total_acc / total_n


# === EMA wrapper ===
class ModelEMA:
    def __init__(self, model, decay=0.9995):
        self.ema = VGG13().to(device)
        self.ema.load_state_dict(model.state_dict(), strict=True)
        for p in self.ema.parameters():
            p.requires_grad_(False)
        self.decay = decay

    @torch.no_grad()
    def update(self, model):
        d = self.decay
        msd = model.state_dict()
        for k, v in self.ema.state_dict().items():
            if v.dtype.is_floating_point:
                v.copy_(v * d + (1.0 - d) * msd[k])

    def state_dict(self):
        return self.ema.state_dict()


# === Run all experiments ===
for cfg in experiments:
    print(f"\n\n=== Running {cfg['name']} ===")

    # Dynamic augmentations
    extra_layers = []
    if cfg.get("extra_aug", False):
        extra_layers += [
            v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1.0)),
            v2.RandomPerspective(distortion_scale=0.5, p=0.5),
        ]

    train_transforms = v2.Compose([
        v2.ToImage(),
        v2.RandomCrop(32, padding=4, padding_mode="reflect"),
        v2.RandomAffine(degrees=10, translate=(0.12, 0.12), scale=(0.9, 1.1)),
        v2.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        *extra_layers,
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean, std, inplace=True),
        v2.RandomErasing(p=0.4, scale=(0.02, 0.15), ratio=(0.3, 3.3), inplace=True),
        v2.RandAugment(num_ops=2, magnitude=cfg["rand_mag"]),
    ])

    # Fresh datasets
    base_train = SVHN_Dataset(train=True, transforms=None)
    val_len = int(len(base_train) * 0.10)
    train_len = len(base_train) - val_len
    train_subset, val_subset = random_split(base_train, [train_len, val_len], generator=torch.Generator().manual_seed(seed))
    train_set = TransformedSubset(train_subset, train_transforms)
    val_set = TransformedSubset(val_subset, eval_transforms)

    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=pin, drop_last=True)
    val_loader = DataLoader(val_set, batch_size=512, shuffle=False, num_workers=NUM_WORKERS, pin_memory=pin)

    # === Model & optimizer ===
    model = VGG13().to(device)
    model = torch.jit.script(model)
    ema = None if cfg.get("remove_ema", False) else ModelEMA(model, decay=cfg["ema_decay"])
    base_lr = cfg["base_lr"]

    optimizer = optim.SGD(
        model.parameters(),
        lr=base_lr,
        momentum=0.9,
        weight_decay=cfg.get("wd", 5e-4),
        nesterov=True,
        fused=(device.type == "cuda")
    )

    # === Scheduler ===
    total_steps = EPOCHS * len(train_loader)
    warmup_steps = max(1, int(total_steps * warmup_pct))

    def lr_lambda(step):
        if step < warmup_steps:
            return float(step + 1) / float(warmup_steps)
        progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
        return min_lr_ratio + (1.0 - min_lr_ratio) * 0.5 * (1.0 + math.cos(math.pi * progress))

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

    # === SWA setup ===
    swa_start_epoch = int(EPOCHS * SWA_START_RATIO)
    swa_model = AveragedModel(model)
    swa_scheduler = SWALR(optimizer, swa_lr=base_lr * 0.05)

    best_val_loss, best_epoch, no_improve = float("inf"), -1, 0
    save_dir = f"/kaggle/working/{cfg['name']}"
    os.makedirs(save_dir, exist_ok=True)

    # === Epoch loop ===
    for epoch in range(1, EPOCHS + 1):
        use_swa = epoch >= swa_start_epoch
        tr_loss, tr_acc = run_epoch(train_loader, model, optimizer, scheduler, ema, criterion, train_mode=True, use_swa=use_swa, mix_p=cfg["mix_p"])
        va_loss, va_acc = run_epoch(val_loader, model, optimizer, scheduler, ema, criterion, train_mode=False, use_swa=use_swa)

        lr_now = optimizer.param_groups[0]["lr"]
        print(f"[{cfg['name']}] Epoch {epoch:03d}/{EPOCHS} | lr={lr_now:.5f} | train_acc={tr_acc*100:.2f}% | val_acc={va_acc*100:.2f}% | val_loss={va_loss:.4f}")

        if use_swa:
            swa_model.update_parameters(model)
            swa_scheduler.step()

        if torch.isnan(torch.tensor(va_loss)) or va_loss == float("inf"):
            print(f"⚠️  NaN validation loss detected in {cfg['name']} — reducing LR.")
            for g in optimizer.param_groups:
                g["lr"] *= 0.5
            continue

        improved = va_loss < best_val_loss - 1e-4
        if improved:
            best_val_loss, best_epoch, no_improve = va_loss, epoch, 0
            torch.save((ema.state_dict() if ema else model.state_dict()), f"{save_dir}/best_vgg13.pth")
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"[{cfg['name']}] Early stopping at epoch {epoch} (best {best_epoch}, val_loss {best_val_loss:.4f}).")
                break

    # === SWA finalize ===
    print(f"Finalizing {cfg['name']} with SWA update...")
    swa_model.train()
    update_bn(train_loader, swa_model)
    swa_model.eval()

    @torch.inference_mode()
    def eval_on_val(module):
        tot_loss, tot_acc, tot_n = 0.0, 0.0, 0
        for inputs, targets in val_loader:
            inputs = ensure_nchw(inputs.to(device, non_blocking=True)).float()
            targets = targets.to(device, non_blocking=True)
            with torch.autocast(device_type=device.type, enabled=enable_half):
                outputs = module(inputs)
                loss = criterion(outputs, targets)
            bs = targets.size(0)
            tot_loss += loss.item() * bs
            tot_acc += accuracy(outputs, targets) * bs
            tot_n += bs
        return tot_loss / tot_n, tot_acc / tot_n

    swa_val_loss, swa_val_acc = eval_on_val(swa_model)
    print(f"[{cfg['name']}] SWA val_loss={swa_val_loss:.4f}, val_acc={swa_val_acc*100:.2f}% | best EMA loss={best_val_loss:.4f}")

    if swa_val_loss < best_val_loss - 1e-4:
        torch.save(swa_model.state_dict(), f"{save_dir}/best_vgg13.pth")
        print(f"[{cfg['name']}] SWA outperformed EMA — saved SWA weights.")
    else:
        print(f"[{cfg['name']}] Keeping EMA weights.")

print("\nAll 4 experiments completed successfully ✅")




=== Running exp1_baseline ===


[exp1_baseline] Epoch 001/100 | lr=0.00754 | train_acc=3.22% | val_acc=0.90% | val_loss=4.8012


[exp1_baseline] Epoch 002/100 | lr=0.01504 | train_acc=10.85% | val_acc=1.40% | val_loss=4.7015


[exp1_baseline] Epoch 003/100 | lr=0.02254 | train_acc=16.24% | val_acc=2.08% | val_loss=4.5687


[exp1_baseline] Epoch 004/100 | lr=0.03004 | train_acc=19.64% | val_acc=4.42% | val_loss=4.4046


...

[exp4_light_aug_fast_lr] Epoch 053/100 | lr=0.06358 | train_acc=76.31% | val_acc=64.32% | val_loss=1.5633


[exp4_light_aug_fast_lr] Epoch 054/100 | lr=0.06168 | train_acc=75.07% | val_acc=64.24% | val_loss=1.5632


[exp4_light_aug_fast_lr] Epoch 055/100 | lr=0.05976 | train_acc=79.97% | val_acc=64.14% | val_loss=1.5682


[exp4_light_aug_fast_lr] Epoch 056/100 | lr=0.05783 | train_acc=73.98% | val_acc=64.40% | val_loss=1.5694
[exp4_light_aug_fast_lr] Early stopping at epoch 56 (best 36, val_loss 1.5311).
Finalizing exp4_light_aug_fast_lr with SWA update...


[exp4_light_aug_fast_lr] SWA val_loss=4.6061, val_acc=0.96% | best EMA loss=1.5311
[exp4_light_aug_fast_lr] Keeping EMA weights.

All 4 experiments completed successfully ✅


In [4]:
# BLOCK 4 — Robust Inference (multi-experiment TTA with BN recalibration and bias debiasing)

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.optim.swa_utils import update_bn

def preprocess_for_model(x: torch.Tensor) -> torch.Tensor:
    x = x.to(device, non_blocking=True)
    if x.ndim == 4 and x.shape[1] != 3 and x.shape[-1] == 3:
        x = x.permute(0, 3, 1, 2).contiguous()
    return x.float()


@torch.inference_mode()
def inference_tta(model):
    """Performs TTA inference with adaptive BN + logit debiasing."""
    # ✅ Recalibrate BatchNorm statistics before inference
    print("↻ Updating BatchNorm stats on test loader for better TTA consistency...")
    update_bn(test_loader, model, device=device)

    model.train()  # allow BN to adapt for each TTA variation
    preds = []

    tta_transforms = [
        v2.Identity(),
        v2.RandomHorizontalFlip(p=1.0),
        v2.RandomAffine(degrees=0, translate=(0.1, 0), scale=None),
        v2.RandomAffine(degrees=0, translate=(0, 0.1), scale=None),
    ]

    for inputs, _ in tqdm(test_loader, leave=False):
        x_raw = inputs
        augmented_logits = []
        for t in tta_transforms:
            # Apply transform safely
            x = torch.stack([t(xi) for xi in x_raw])
            x = preprocess_for_model(x)

            # ✅ Force FP32 precision for stable inference
            with torch.no_grad():
                logits = model(x)

                # ✅ Debias the final linear layer
                last_linear = next((mod for mod in reversed(list(model.modules())) if isinstance(mod, nn.Linear)), None)
                if last_linear is not None and hasattr(last_linear, "bias") and last_linear.bias is not None:
                    bias_vec = last_linear.bias.detach().view(1, -1)
                    logits = logits - bias_vec

                augmented_logits.append(logits)

        # Average predictions from all TTA views
        avg_logits = torch.stack(augmented_logits).mean(dim=0)
        preds.extend(torch.argmax(avg_logits, dim=1).tolist())

    model.eval()
    return preds


# === Run inference for all experiments ===
for cfg in experiments:
    exp_dir = f"/kaggle/working/{cfg['name']}"
    best_path = f"{exp_dir}/best_vgg13.pth"

    if not os.path.exists(best_path):
        print(f"[{cfg['name']}] ⚠️ Skipped — no best_vgg13.pth found.")
        continue

    print(f"\n=== Inference for {cfg['name']} ===")

    # ✅ Load model safely in FP32
    model = VGG13().to(device).float()
    state_dict = torch.load(best_path, map_location=device)
    model.load_state_dict(state_dict, strict=True)
    model.eval()

    # ✅ Optional TorchScript compilation
    scripted = torch.jit.script(model)
    scripted_path = f"{exp_dir}/best_vgg13_scripted.pt"
    scripted.save(scripted_path)
    print(f"[{cfg['name']}] Saved scripted model → {scripted_path}")

    torch.cuda.empty_cache()

    # ✅ Run robust TTA inference
    preds = inference_tta(model)

    # 🔍 Fallback check: if model collapsed to single-class output
    vals, cnts = np.unique(np.array(preds), return_counts=True)
    if len(vals) == 1:
        print(f"⚠️ Warning: model predicted only one class ({vals[0]}). Check training stability.")
        # Try disabling bias correction and re-run once
        preds = []
        model.eval()
        for inputs, _ in tqdm(test_loader, leave=False):
            x = preprocess_for_model(inputs)
            with torch.no_grad():
                logits = model(x)
            preds.extend(torch.argmax(logits, dim=1).tolist())
        vals, cnts = np.unique(np.array(preds), return_counts=True)

    # === Log results ===
    print(f"[{cfg['name']}] Sample preds (first 20): {preds[:20]}")
    hist = list(zip(vals[:10].tolist(), cnts[:10].tolist()))
    print(f"[{cfg['name']}] Pred histogram (top 10): {hist}")

    # === Save submission ===
    out_csv = f"{exp_dir}/submission.csv"
    pd.DataFrame({"ID": list(range(len(preds))), "target": preds}).to_csv(out_csv, index=False)
    print(f"[{cfg['name']}] Saved submission → {out_csv}")

print("\n✅ All experiment inferences completed successfully with stable TTA and BN recalibration.")



=== Inference for exp1_baseline ===
[exp1_baseline] Saved scripted model → /kaggle/working/exp1_baseline/best_vgg13_scripted.pt
↻ Updating BatchNorm stats on test loader for better TTA consistency...


[exp1_baseline] Sample preds (first 20): [98, 50, 61, 6, 7, 33, 15, 21, 0, 98, 28, 25, 60, 27, 3, 5, 19, 57, 83, 40]
[exp1_baseline] Pred histogram (top 10): [(0, 99), (1, 92), (2, 115), (3, 114), (4, 93), (5, 107), (6, 94), (7, 100), (8, 96), (9, 93)]
[exp1_baseline] Saved submission → /kaggle/working/exp1_baseline/submission.csv

=== Inference for exp2_strong_aug ===
[exp2_strong_aug] Saved scripted model → /kaggle/working/exp2_strong_aug/best_vgg13_scripted.pt
↻ Updating BatchNorm stats on test loader for better TTA consistency...


[exp2_strong_aug] Sample preds (first 20): [98, 50, 61, 6, 59, 33, 15, 21, 94, 98, 28, 41, 60, 27, 3, 83, 19, 57, 83, 40]
[exp2_strong_aug] Pred histogram (top 10): [(0, 100), (1, 74), (2, 103), (3, 132), (4, 97), (5, 120), (6, 95), (7, 85), (8, 94), (9, 94)]
[exp2_strong_aug] Saved submission → /kaggle/working/exp2_strong_aug/submission.csv

=== Inference for exp3_fast_opt ===
[exp3_fast_opt] Saved scripted model → /kaggle/working/exp3_fast_opt/best_vgg13_scripted.pt
↻ Updating BatchNorm stats on test loader for better TTA consistency...


[exp3_fast_opt] Sample preds (first 20): [98, 50, 61, 6, 64, 33, 18, 21, 98, 98, 28, 41, 60, 27, 3, 5, 19, 57, 83, 40]
[exp3_fast_opt] Pred histogram (top 10): [(0, 99), (1, 86), (2, 100), (3, 113), (4, 101), (5, 108), (6, 94), (7, 89), (8, 100), (9, 82)]
[exp3_fast_opt] Saved submission → /kaggle/working/exp3_fast_opt/submission.csv

=== Inference for exp4_light_aug_fast_lr ===
[exp4_light_aug_fast_lr] Saved scripted model → /kaggle/working/exp4_light_aug_fast_lr/best_vgg13_scripted.pt
↻ Updating BatchNorm stats on test loader for better TTA consistency...


[exp4_light_aug_fast_lr] Sample preds (first 20): [1, 50, 61, 6, 59, 33, 18, 9, 98, 98, 28, 41, 60, 27, 3, 5, 19, 57, 83, 40]
[exp4_light_aug_fast_lr] Pred histogram (top 10): [(0, 90), (1, 95), (2, 100), (3, 123), (4, 87), (5, 110), (6, 93), (7, 76), (8, 115), (9, 97)]
[exp4_light_aug_fast_lr] Saved submission → /kaggle/working/exp4_light_aug_fast_lr/submission.csv

✅ All experiment inferences completed successfully with stable TTA and BN recalibration.
